In [1]:
from dotenv import load_dotenv
from os import environ
from databases import Database
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
load_dotenv('.env')
DATABASE_URL = environ.get('DB_CONNECTION')+'://'+environ.get('DB_USERNAME')+':'+environ.get('DB_PASSWORD')+'@'+environ.get('DB_HOST')+':'+environ.get('DB_PORT')+'/'+environ.get('DB_DATABASE')
database = Database(DATABASE_URL)

In [3]:
hotels_df = pd.read_csv('../archive/Hotel_details.csv')
rooms_df = pd.read_csv('../archive/hotels_RoomPrice.csv')

In [4]:
cities = hotels_df['city']
cities = cities.drop_duplicates()
cities = cities.reset_index()
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25693 entries, 0 to 25692
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   25693 non-null  int64 
 1   city    25693 non-null  object
dtypes: int64(1), object(1)
memory usage: 401.6+ KB


In [6]:
await database.connect()
query = "INSERT INTO cities(id, name, image) VALUES"
image_link = 'https://unsplash.com/photos/Nyvq2juw4_o/download?ixid=MnwxMjA3fDB8MXxzZWFyY2h8Mnx8Y2l0eXxlbnwwfHx8fDE2NjUwMDAzNjM&force=true&w=640'
for index, city in cities.iterrows():
    query = query + "("+str(index)+", '"+city['city'].replace("'", "''")+"','"+image_link+"'),"
query = query[:-1]
print("The query was created")
await database.execute(query=query)

The query was created


In [35]:
hotels = hotels_df[['hotelid', 'hotelname', 'starrating', 'address', 'city']]
hotels = hotels.dropna()
hotels = hotels.drop_duplicates(subset=["hotelid"], keep='last')
hotels = hotels.reset_index()
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101103 entries, 0 to 101102
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   index       101103 non-null  int64 
 1   hotelid     101103 non-null  int64 
 2   hotelname   101103 non-null  object
 3   starrating  101103 non-null  int64 
 4   address     101103 non-null  object
 5   city        101103 non-null  object
dtypes: int64(3), object(3)
memory usage: 4.6+ MB


In [36]:
load_in_each_query = 10000
for x in range(0, hotels['index'].count(), load_in_each_query):
    query = "INSERT INTO hotels(id, name, address, image, credit, star, city_id) VALUES "
    for index, hotel in hotels.iterrows():
        if x<=index<x+load_in_each_query:
            query = query + "("+str(hotel['hotelid'])+", '"+hotel['hotelname'].replace("'", "''")+"', '"+hotel['address'].replace("'", "''")+"', 'image_link', 0, "+str(hotel["starrating"])+", "+str(cities.loc[cities['city']==hotel['city']].index[0])+"),"
    query = query[:-1]
    await database.execute(query=query)

In [37]:
rooms = rooms_df[['id', 'onsiterate', 'maxoccupancy', 'hotelcode']]
rooms = rooms.dropna()
rooms = rooms.reset_index()
rooms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165873 entries, 0 to 165872
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   index         165873 non-null  int64  
 1   id            165873 non-null  int64  
 2   onsiterate    165873 non-null  float64
 3   maxoccupancy  165873 non-null  int64  
 4   hotelcode     165873 non-null  int64  
dtypes: float64(1), int64(4)
memory usage: 6.3 MB


In [40]:
load_in_each_query = 10000
for x in range(0, rooms['index'].count(), load_in_each_query):
    query = "INSERT INTO rooms(id, number, price, max_capacity, floor, hotel_id) VALUES "
    for index, room in rooms.iterrows():
        if x<=index<x+load_in_each_query:
            query = query + "("+str(index)+", '"+str(int(room['id']))+"', "+str(int(room['onsiterate']))+", "+str(room["maxoccupancy"])+", 1, 8992131),"
    query = query[:-1]
    print("The Query Was Created")
    await database.execute(query=query)

The Query Was Created
The Query Was Created
The Query Was Created
The Query Was Created
The Query Was Created
The Query Was Created
The Query Was Created
The Query Was Created
The Query Was Created
The Query Was Created
The Query Was Created
The Query Was Created
The Query Was Created
The Query Was Created
The Query Was Created
The Query Was Created
The Query Was Created
